In [21]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from pathlib import Path


In [22]:
#Loading file into Path
data = Path('frmgham2.csv')
#Reading data file
data_df = pd.read_csv(data)
data_df.head()


,RANDID,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,...,CVD,HYPERTEN,TIMEAP,TIMEMI,TIMEMIFC,TIMECHD,TIMESTRK,TIMECVD,TIMEDTH,TIMEHYP
0,2448,1,195.0,39,106.0,70.0,0,0.0,26.97,0,...,1,0,8766,6438,6438,6438,8766,6438,8766,8766
1,2448,1,209.0,52,121.0,66.0,0,0.0,NaN,0,...,1,0,8766,6438,6438,6438,8766,6438,8766,8766
2,6238,2,250.0,46,121.0,81.0,0,0.0,28.73,0,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
3,6238,2,260.0,52,105.0,69.5,0,0.0,29.43,0,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
4,6238,2,237.0,58,108.0,66.0,0,0.0,28.50,0,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766


In [23]:
# Counting our participants using nunique
data_df['RANDID'].nunique()

#Filtering data for the columns listed below
data_df = data_df[['RANDID', 'SEX', 'TOTCHOL', 'AGE','CURSMOKE',
       'CIGPDAY',  'educ','PREVCHD', 'PERIOD', 'BMI']]

#Dropping data rows containing NaN values
data_df = data_df.dropna()
data_df.head()


,RANDID,SEX,TOTCHOL,AGE,CURSMOKE,CIGPDAY,educ,PREVCHD,PERIOD,BMI
0,2448,1,195.0,39,0,0.0,4.0,0,1,26.97
2,6238,2,250.0,46,0,0.0,2.0,0,1,28.73
3,6238,2,260.0,52,0,0.0,2.0,0,2,29.43
4,6238,2,237.0,58,0,0.0,2.0,0,3,28.50
5,9428,1,245.0,48,1,20.0,1.0,0,1,25.34


In [24]:
#Grouping dataframe by ID to identify periods
databyperiod = data_df.groupby('RANDID').count()
databyperiod.head()

,SEX,TOTCHOL,AGE,CURSMOKE,CIGPDAY,educ,PREVCHD,PERIOD,BMI
RANDID,,,,,,,,,
2448,1,1,1,1,1,1,1,1,1
6238,3,3,3,3,3,3,3,3,3
9428,2,2,2,2,2,2,2,2,2
10552,2,2,2,2,2,2,2,2,2
11252,2,2,2,2,2,2,2,2,2


In [25]:
#Filtering our participants into list only those who completed all 3 periods of the study 
nonull = databyperiod.loc[databyperiod['PERIOD'] == 3, :]
#Assigning index list to a variable
column_ID = nonull.index
#Initialzing List
column_list = []
#Running through for loop of column_ID values to append IDs to column_list
for ID in column_ID: 
    column_list.append(ID)

In [26]:
#Initiliazing New_data_df as an object
new_data_df = data_df

#Running for loop to select each row of data that matched the column list
for ID in column_list:
    filtered_df =  data_df.loc[data_df['RANDID'] == ID, :]

    #Concated the data frame arg 1 is where you want to store it and arg2 is where you want the data from 
    new_data_df = pd.concat([new_data_df, filtered_df])

new_data_df.head()


,RANDID,SEX,TOTCHOL,AGE,CURSMOKE,CIGPDAY,educ,PREVCHD,PERIOD,BMI
0,2448,1,195.0,39,0,0.0,4.0,0,1,26.97
2,6238,2,250.0,46,0,0.0,2.0,0,1,28.73
3,6238,2,260.0,52,0,0.0,2.0,0,2,29.43
4,6238,2,237.0,58,0,0.0,2.0,0,3,28.50
5,9428,1,245.0,48,1,20.0,1.0,0,1,25.34


In [27]:
#Renaming Columns 
new_data_df = new_data_df.rename(columns={
        "RANDID" : "ID",
        "TOTCHOL" : "TOTAL CHOLESTROL",
        "CURSMOKE" : "SMOKING STATUS",
        "CIGPDAY" : "CIGS PER DAY",
        "educ" : "EDUCATION",
        "PREVCHD" : "PREVALENT CHD"
                                }                                 
                                 )

In [28]:
#Replacing values for sex, prevalent CHD, and smoking status to make it more readable
new_data_df['SEX'] = np.where(new_data_df['SEX'] == 1, 'M', 'F')
new_data_df['SMOKING STATUS'] = np.where(new_data_df['SMOKING STATUS'] == 0, 'NO', 'YES')
new_data_df['PREVALENT CHD'] = np.where(new_data_df['PREVALENT CHD'] == 0, 'NO', 'YES')
new_data_df.head()

,ID,SEX,TOTAL CHOLESTROL,AGE,SMOKING STATUS,CIGS PER DAY,EDUCATION,PREVALENT CHD,PERIOD,BMI
0,2448,M,195.0,39,NO,0.0,4.0,NO,1,26.97
2,6238,F,250.0,46,NO,0.0,2.0,NO,1,28.73
3,6238,F,260.0,52,NO,0.0,2.0,NO,2,29.43
4,6238,F,237.0,58,NO,0.0,2.0,NO,3,28.50
5,9428,M,245.0,48,YES,20.0,1.0,NO,1,25.34


In [29]:
#Converting CIGS PER DAY & EDUCATION to integers
CIGS_PER_DAY = new_data_df['CIGS PER DAY'].astype('Int64')
new_data_df['CIGS PER DAY'] = CIGS_PER_DAY.values
EDUCATION = new_data_df['EDUCATION'].astype('Int64')
new_data_df['EDUCATION'] = EDUCATION.values

new_data_df.head()

,ID,SEX,TOTAL CHOLESTROL,AGE,SMOKING STATUS,CIGS PER DAY,EDUCATION,PREVALENT CHD,PERIOD,BMI
0,2448,M,195.0,39,NO,0,4,NO,1,26.97
2,6238,F,250.0,46,NO,0,2,NO,1,28.73
3,6238,F,260.0,52,NO,0,2,NO,2,29.43
4,6238,F,237.0,58,NO,0,2,NO,3,28.50
5,9428,M,245.0,48,YES,20,1,NO,1,25.34


In [30]:
#Saving cleaned data into a new csv file 
new_data_df.to_csv("clean_data.csv", index = False)

#Test